## LangChain Code Exe

In [1]:
from langchain_openai.llms import OpenAI
from langchain_experimental.utilities import PythonREPL
from langchain.tools import Tool
from langchain.agents import AgentType, initialize_agent
from settings import OPENAI_API_KEY

llm = OpenAI(openai_api_key=OPENAI_API_KEY)
python_repl = PythonREPL()
tool = Tool(name="python_repl",
            description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
            func=python_repl.run,)
agent = initialize_agent(llm=llm, tools=[tool, ], agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
result = agent.invoke("What are the prime numbers until 20?")
result

/var/folders/b0/8785kc0j0nb0k5qm2r5h8nyh0000gn/T/ipykernel_35374/4276959909.py:12: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(llm=llm, tools=[tool, ], agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)




> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 Create a list of numbers from 2 to 20 and check if each number is prime
Action: python_repl
Action Input: numbers = list(range(2,21))
Observation: 
Thought: Use a for loop to iterate through the numbers list and check if each number is prime
Action: python_repl
Action Input: for num in numbers:
Observation: IndentationError("expected an indented block after 'for' statement on line 1", ('<string>', 1, 20, 'for num in numbers:\n', 1, -1))
Thought: Need to write a function to check if a number is prime
Action: python_repl
Action Input: def is_prime(n):\n\tif n < 2:\n\t\treturn False\n\tfor i in range(2, n):\n\t\tif n % i == 0:\n\t\t\treturn False\n\treturn True
Observation: SyntaxError('unexpected character after line continuation character', ('<string>', 1, 18, 'def is_prime(n):\\n\\tif n < 2:\\n\\t\\treturn False\\n\\tfor i in range(2, n):\\n\\t\\tif n % i == 0:\\n\\t\\t\\treturn False\\n\\treturn True\n', 1, 0))
Thought: Need to fix the syntax error by removing the line continuation c

{'input': 'What are the prime numbers until 20?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

## MetaGPT agent simulation

[github](https://github.com/geekan/MetaGPT/tree/main)

In [ ]:
import asyncio
from metagpt.roles import (
    Architect,
    Engineer,
    ProductManager,
    ProjectManager,
)
from metagpt.team import Team

async def startup(idea: str):
    company = Team()
    company.hire(
        [
            ProductManager(),
            Architect(),
            ProjectManager(),
            Engineer(),
        ]
    )
    company.invest(investment=3.0)
    company.run_project(idea=idea)

    await company.run(n_round=5)

await startup(idea="write a cli blackjack game")

## PlanAndExecute chain

In [3]:
from langchain_experimental.plan_and_execute import load_agent_executor, load_chat_planner, PlanAndExecute

planner = load_chat_planner(llm)
executor = load_agent_executor(llm=llm, tools=[tool], verbose=True)
agent_executor = PlanAndExecute(planner=planner,
                                executor=executor,
                                verbose=True,
                                handle_parsing_errors="Check your output and make sure it conforms!",
                                return_intermediate_steps=True)
agent_executor.invoke("Write a tetris game in python!")



> Entering new PlanAndExecute chain...
steps=[Step(value='Create a 2D grid of squares to represent the game board.'), Step(value='Define the shapes and their rotations using lists or dictionaries.'), Step(value='Randomly generate a new shape and display it on the game board.'), Step(value='Allow the user to move the shape left or right using keyboard inputs.'), Step(value='Implement a function to check for collisions with the edges of the game board and other shapes.'), Step(value='Allow the user to rotate the shape using keyboard inputs.'), Step(value='Implement a function to check for collisions with other shapes when rotating.'), Step(value='Create a scoring system based on completed rows.'), Step(value='Display the current score and game board on the screen.'), Step(value='Continue generating and moving shapes until the game board is full.'), Step(value='End the game when the game board is full and display the final score.\n')]

> Entering new AgentExecutor chain...
Thought: I ca

{'input': 'Write a tetris game in python!',
 'output': 'Human:\nThought: I can modify the while loop to check if the game board is full after each move. If it is, I can print out the final score and end the game.\nAction:\n{\n  "action": "python_repl",\n  "action_input": "def main():\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\n    global grid, score\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\n    grid = [[\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\' \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'] * 10] * 10\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\n    score = 0\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\n    while not all(\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'\\\\\\\\\\\\\\\\\\\\\

## code generation
One thing we need is clear instructions for a language model to write Python code in a certain  form – we can reference syntax guidelines,

- Choose Model that optimized for producing software code specifically. like：Codex, PythonCoder, and  AlphaCode are designed for code generation capabilities.
- Set Model
    - longer context,  so we don’t get cut off in the middle of a function
    - low temperature, so it doesn’t get too wild.

-  Give Feedback: execute the code to test it and provide meaningful feedback to the LLM.
-  LangChain tools argument to the executor allows specifying Python modules, libraries, and other resources that can extend the LLM’s reach. 
    > For example, we can use tools to write the code to file,  execute it with different inputs, capture the outputs, check correctness, analyze style, and more.  Based on the tool outputs, we can provide feedback to the LLM on which parts of the code worked  and which need improvement. The LLM can then generate enhanced code incorporating this  feedback.

### Non-Tool

In [6]:
from langchain.prompts import PromptTemplate
from langchain import LLMChain

DEV_PROMPT = """
            You are a software engineer who writes Python code given tasks or objectives.
            Come up with a python code for this task: {task}
            Please use PEP8 syntax and comments!
            """ 
software_prompt = PromptTemplate.from_template(DEV_PROMPT)
llm = OpenAI(temperature=0, max_tokens=1000)
software_llm = software_prompt | llm

In [ ]:
### With Tools

In [ ]:
"""Python Sandbox Developer."""
import io
import logging
import os
import uuid
from contextlib import contextmanager, redirect_stdout
from logging import FileHandler
from pathlib import Path
from typing import Literal

import pip
from langchain import LLMChain, PromptTemplate
from langchain.chains.base import Chain
from langchain.llms import FakeListLLM
from langchain_experimental.tools.python.tool import sanitize_input
from pip._internal.exceptions import InstallationError
from pydantic import BaseModel, Field

logging.basicConfig(encoding="utf-8", level=logging.INFO)
DEV_PROMPT = (
    "You are a software engineer who writes Python code given tasks or objectives. "
    "Come up with a python code for this task: {task}"
    "Please use PEP8 syntax and comments!"
)


class PythonExecutorInput(BaseModel):
    code: str = Field()


def meaningful_output(func):
    def wrapper(*args, **kwargs):
        func_output = func(*args, **kwargs)
        print(func_output)
        if len(func_output.strip()) > 0:
            return f"The code returned the following:\n" \
                   f"{func_output}"
        else:
            return "The code returned nothing."
    return wrapper


@contextmanager
def set_directory(path: Path):
    """Sets working directory within the context

    Parameters:
        * path (Path): path to directory
    """
    origin = Path().absolute()
    try:
        os.chdir(path)
        yield
    finally:
        os.chdir(origin)


class PythonDeveloper:
    """Execution environment for Python code."""

    def __init__(
            self,
            llm_chain: Chain,
            path: str = "dev",
            audit_file: str = "audit.log",
            do_sanitize_input: bool = True,
            save_intermediate_steps: bool = False
    ):
        self.save_intermediate_steps = save_intermediate_steps
        self.llm_chain = llm_chain
        self.path = path
        self.create_directory()
        self.logger = logging.getLogger()
        self.logger.addHandler(
            self.setup_audit_trail(audit_file=audit_file)
        )
        self.do_sanitize_input = do_sanitize_input

    def write_code(self, task: str) -> str:
        """Given a task description write Python code.

        If intermediate steps are desired, store the code to
        a separate Python file.
        """
        code = self.llm_chain.run(task)
        if self.save_intermediate_steps:
            self.write_file("", code, "w")
        return code

    @staticmethod
    def setup_audit_trail(audit_file: str) -> FileHandler:
        """Set up a logger that tracks all calls to run."""
        formatter = logging.Formatter(
            "%(asctime)s | %(levelname)s | %(message)s"
        )
        file_handler = logging.FileHandler(audit_file)
        file_handler.setLevel(logging.DEBUG)
        file_handler.setFormatter(formatter)
        return file_handler

    def install_package(
            self,
            module_not_found_error: ModuleNotFoundError
    ) -> bool:
        """Install a package.

        Returns True if installation successful.
        Note: We could implement a logic for virtual environments here.
        """
        try:
            package = str(module_not_found_error).strip().split(" ")[-1].strip("'")
            self.logger.info(f"installing {package}")
            pip.main(['install', package])
            return True
        except InstallationError as ex:
            # Any other error, we want to fail here.
            self.logger.exception(ex)
            return False

    @meaningful_output
    def run(
            self,
            task: str,
            filename: str = "main.py",
            mode: Literal["w", "a"] = "w"
    ) -> str:
        """Generate and execute Python code.

        Returns the output from the run.
        """
        self.logger.info(f"Task:\n{task}")
        code = self.write_code(task)
        self.logger.info(f"Code:\n{code}")
        if self.do_sanitize_input:
            code = sanitize_input(code)

        self.write_file(code=code, filename=filename, mode=mode)
        # import executor; we can try others like pylint
        try:
            # with DirectorySandbox(self.path):
            return self.execute_code(code, filename)
        except (ModuleNotFoundError, NameError) as ex:
            return str(ex)
        except SyntaxError as ex:
            return f"This is not valid Python code! Exception thrown: {ex}"
        except FileNotFoundError as ex:
            # If this is an image, we could add a tool to create images.
            return f"This file doesn't exist!\n{ex}"
        except SystemExit as ex:
            self.logger.warning(ex)
            return str(ex)

    def execute_code(self, code: str, filename: str) -> str:
        """Execute a python code."""
        try:
            with set_directory(Path(self.path)):
                ns = dict(__file__=filename, __name__="__main__")
                function = compile(code, "<>", "exec")
                with redirect_stdout(io.StringIO()) as f:
                    exec(function, ns)
                    return f.getvalue()
        except ModuleNotFoundError as ex:
            if self.install_package(ex):
                return self.execute_code(code, filename)
            raise ex

    def write_file(
            self,
            filename: str,
            code: str,
            mode: Literal["w", "a"] = "w"
    ) -> Path:
        """Write code to disk.

        If filename is an empty string, write to
        random filename. This can be useful as
        intermediate step.

        Returns the path to the new file.
        """
        if not filename:
            filename = str(uuid.uuid4()) + ".py"
        fullpath = Path(self.path) / filename
        with open(fullpath, mode, encoding="utf-8") as f:
            f.write(code)

        return fullpath

    def create_directory(self):
        """Create a directory for the project."""
        os.makedirs(self.path, exist_ok=True)


if __name__ == "__main__":
    software_prompt = PromptTemplate.from_template(DEV_PROMPT)
    # careful: if you have the wrong model spec, you might not get any code!
    software_llm = LLMChain(
        llm=FakeListLLM(
            responses=[
                "import os; print(os.getcwd())",
                "import os; os.listdir('.')",
                "print('hello world!')"
            ]
        ),
        prompt=software_prompt
    )
    env = PythonDeveloper(software_llm)